# Full Test-Ledoit2017 Table 1

In [20]:
import numpy as np
import pandas as pd
import nonlinshrink as nls

def showmatrixinfo(matrix):
    print("Matrix shape: ",np.shape(matrix))
    print(matrix)

## Data selection

## Empirical test

### Functions & data

In [65]:
## Functions:


def test_result(omega,method):
    
    ### Use estimated covariance matrix to calculate w
    Omega= np.matrix(omega)
    w=(Omega.I@np.ones((100,1)))/(np.ones((1,100))@Omega.I@np.ones((100,1)))
    print("\n"+method+":")
    print("weight_vector:")
    print(np.shape(w))
    
    ###########calculate payoff and excess payoff(ri-rf)
    
    payoff=w.T@yt_test*100
    
    exc_payoff=payoff-rf_daily
    print(np.shape(exc_payoff))
    
    
    return exc_payoff.tolist()[0]

In [53]:
#Load and check data

yt_est=np.load("Fix100Stock_est_matrix.npy")
yt_test=np.load("Fix100Stock_test_matrix.npy")

FFdata = pd.read_csv('F-F_Research_Data_Factors_daily_Fix100.csv')
rf_rate=FFdata.loc[250:,"RF"].to_numpy()
rf_daily=rf_rate/250                          

data=[yt_est,yt_test,FFdata]

print(np.shape(yt_est))               
print(np.shape(yt_test)) 
print(np.shape(rf_daily))
print(yt_test)

(100, 250)
(100, 21)
(21,)
[[ 0.01789964 -0.01992981  0.00358893 ... -0.01463477  0.03217799
   0.01199075]
 [-0.01161362 -0.01051343  0.02499983 ...  0.00678584  0.02267097
   0.02119442]
 [-0.00213677 -0.00535299  0.01399348 ...  0.0042062  -0.00711211
  -0.01652012]
 ...
 [-0.00507116  0.00111554  0.02228419 ...  0.00120744  0.03679161
   0.01803369]
 [ 0.          0.00722904  0.01435389 ...  0.00668137  0.0176998
   0.0021737 ]
 [-0.01433089 -0.00484653  0.01623402 ... -0.03137801  0.02676057
   0.03017811]]


### Methods:

In [109]:
def run_1N():
    N=100
    omega=np.identity(100)

    return1N=test_result(omega,"1/N")
    return return1N

def run_Samp():
    ## ---------Sample Covariance Estimation
    omega=np.cov(yt_est)
    print("Matrix shape:")
    print(np.shape(omega))

    returnSamp=test_result(omega,"Samp")
    return returnSamp
    
def run_Lin():
    ##--------Lin(Ledoit-Wolf 2004 methpd) shrinkage Estimation
    from sklearn.covariance import ledoit_wolf

    omega=ledoit_wolf(yt_est.T)[0]

    returnLin=test_result(omega,"Lin")
    return returnLin

##-------NonLin shrinkage Estimation--------------

def run_NonLin():
    import nonlinshrink as nls

    omega=nls.shrink_cov(yt_est.T)
    print(np.shape(omega))

    returnNonLin=test_result(omega,"NonLin")
    return returnNonLin

##--------Single factor estimation---------------------##

def run_SF():
        ##   Generate Factor
    equalw=np.array([[0.01]*100])
    print("equal portfolio: ",np.shape(equalw))
    factor=equalw@yt_est
    print("Factor matrix shape: ",np.shape(factor))


    ##estimate SigmaF
    var_f=np.var(factor,ddof=1)      #variance of factor,

    np.savetxt("SF_factor.csv",factor,delimiter=",")

    ##compute cov(Ri,Rf),the covariance of stocks and factor
    var_if=np.cov(yt_est,factor)[-1,:-1]
    var_if=np.matrix(var_if)          #convert to 1X100 matrix


    SigmaSF=var_if.T*var_if/var_f  
    for i in range(100):
        SigmaSF[i,i]=np.cov(yt_est)[i,i]


    showmatrixinfo(SigmaSF)    ####Seems correct

    returnSF=test_result(SigmaSF,"SF")
    return returnSF


##-----------------FAMA FRENCH estimation------------------------

def run_FF():
    #####First, Generate 3-factors array.


    from sklearn.linear_model import LinearRegression

    LG=LinearRegression()

    LG.fit(FFfactors,yt_est.T)      ##FFfactor matirx is a (250,3) matrix!
    betas=LG.coef_
    print("Beta matrix: ",np.shape(betas))

    var_ff=np.cov(FFfactors.T)      ##Covariance of FAMA FRENCH 3 Factor model.

    SigmaF=betas@var_ff@betas.T

    ###As same as SF, the diagonal need add residual,or replace by var(Ri)

    for i in range(100):
        SigmaF[i,i]=np.cov(yt_est)[i,i]

    print("SigmaF: ")
    showmatrixinfo(SigmaF)

    returnFF=test_result(SigmaF,"FF")
    return returnFF



def run_POET():

#---------POET estimation-----------------#
    from sklearn.decomposition import PCA
    
    print("generating components:")
    pca = PCA(n_components=5, copy=True)
    pca.fit(yt_est)
    factors=pca.components_
    print('PCA Componets:\n', pca.components_)
    print(np.shape(pca.components_))
    print('\nEigenvalues:', pca.explained_variance_)
    print('Variance explaination:', pca.explained_variance_ratio_)
    print("Add up: ",np.sum(pca.explained_variance_ratio_))


    ##Regression:
    from sklearn.linear_model import LinearRegression

    print("\nRegression:")
    LG2=LinearRegression()

    LG2.fit(factors.T,yt_est.T)
    betas=LG2.coef_
    print("Beta matrix: ",np.shape(betas))

    var_fs=np.cov(factors)

    SigmaF=betas@var_fs@betas.T

    ###As same as SF, the diagonal need add residual,or replace by var(Ri)

    for i in range(100):
        SigmaF[i,i]=np.cov(yt_est)[i,i]

    showmatrixinfo(SigmaF)

    returnPOET=test_result(SigmaF,"POET")
    return returnPOET


def run_NLSF():
    ## NL-SF
    eigenvalue, eigenvectors = np.linalg.eig(SigmaSF)
    print(np.shape(eigenvalue))
    print(np.shape(eigenvectors))

    print(np.allclose(SigmaSF,eigenvectors@np.diag(eigenvalue)@eigenvectors.T))

    diag=np.identity(100)
    diag2=np.zeros((100,100))
    for i in range(100):
        diag[i,i]=pow(eigenvalue[i],-1/2)
        diag2[i,i]=pow(eigenvalue[i],1/2)
    ##Generate Yt x Sigma_SF to the power of -1/2
    SigmaSF2=eigenvectors@diag@eigenvectors.T  ##(1/2)
    SigmaSF3=eigenvectors@diag2@eigenvectors.T  ##(-1/2)
    SigmaC_hat=nls.shrink_cov(yt_est.T@SigmaSF2)

    #Reincorporating the structure.
    SigmaNLSF=SigmaSF3@SigmaC_hat@SigmaSF3
    returnNLSF=test_result(SigmaNLSF,"NL-SF")
    return returnNLSF


### Running test

In [142]:
return1N=[]
returnSamp=[]
returnLin=[]
returnNonLin=[]
returnSF=[]
returnFF=[]
returnPOET=[]
returnNLSF=[]

def run(data):
    yt_est=data[0]
    yt_test=data[1]
    FFdata=data[2]
    
    return1N.extend(run_1N())
    returnSamp.extend(run_Samp())
    returnLin.extend(run_Lin())
    returnNonLin.extend(run_NonLin())
    returnSF.extend(run_SF())
    returnFF.extend(run_FF())
    returnPOET.extend(run_POET())
    returnNLSF.extend(run_NLSF())
    
    return

run(data)

ret_mat=np.matrix([return1N,
returnSamp,
returnLin,
returnNonLin,
returnSF,
returnFF,
returnPOET,
returnNLSF])

print("Return maxtrix:")
print(np.shape(ret_mat))



1/N:
weight_vector:
(100, 1)
(1, 21)
Matrix shape:
(100, 100)

Samp:
weight_vector:
(100, 1)
(1, 21)

Lin:
weight_vector:
(100, 1)
(1, 21)
(100, 100)

NonLin:
weight_vector:
(100, 1)
(1, 21)
equal portfolio:  (1, 100)
Factor matrix shape:  (1, 250)
Matrix shape:  (100, 100)
[[4.65997962e-04 1.82112016e-04 1.64998702e-04 ... 1.01471267e-04
  7.28569689e-05 1.56005464e-04]
 [1.82112016e-04 5.05786101e-04 1.92200615e-04 ... 1.18199960e-04
  8.48682692e-05 1.81724739e-04]
 [1.64998702e-04 1.92200615e-04 3.68006616e-04 ... 1.07092549e-04
  7.68930823e-05 1.64647818e-04]
 ...
 [1.01471267e-04 1.18199960e-04 1.07092549e-04 ... 4.23508458e-04
  4.72878781e-05 1.01255480e-04]
 [7.28569689e-05 8.48682692e-05 7.68930823e-05 ... 4.72878781e-05
  2.24052101e-04 7.27020323e-05]
 [1.56005464e-04 1.81724739e-04 1.64647818e-04 ... 1.01255480e-04
  7.27020323e-05 5.87914599e-04]]

SF:
weight_vector:
(100, 1)
(1, 21)
Beta matrix:  (100, 3)
SigmaF: 
Matrix shape:  (100, 100)
[[4.65997962e-04 1.81657701e-

### Ploting results

In [140]:
#####Show all the results

AV=np.mean(ret_mat,axis=1)*250
AV=AV.T.tolist()[0]
    
SD=np.std(ret_mat,axis=1)*pow(250,.5)
SD=SD.T.tolist()[0]

SR=[]
for i in range(len(SD)):
    SR.append(AV[i]/SD[i])





def prtb():
    print("Table 1")
    print("Performance measures for various estimators of the GMV portfolio")
    print("{}".format('''Period: January 19, 1973 to December 31, 2011" '''))
    print("\t1/N \tSample \tLin \tNolin \tSF \tFF \tPOET \tNL-SF")
    print("{:-^70}".format(""))
    print("{:^70}".format("N=100"))
    print("{:-^70}".format(""))
    print("AV \t{:.2f} \t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}".format(
        AV[0],AV[1],AV[2],AV[3],AV[4],AV[5],AV[6],AV[7])
    )
    print("SD \t{:.2f} \t{:.2f}\t{:.2f}\t{:.2f}\t{:.1f}\t{:.2f}\t{:.2f}\t{:.2f}".format(
        SD[0],SD[1],SD[2],SD[3],SD[4],SD[5],SD[6],SD[7]))
    print("SR \t{:.2f} \t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}".format(
        SR[0],SR[1],SR[2],SR[3],SR[4],SR[5],SR[6],SR[7]))
        
    return


prtb()

Table 1
Performance measures for various estimators of the GMV portfolio
Period: January 19, 1973 to December 31, 2011" 
	1/N 	Sample 	Lin 	Nolin 	SF 	FF 	POET 	NL-SF
----------------------------------------------------------------------
                                N=100                                 
----------------------------------------------------------------------
AV 	54.87 	55.60	61.44	73.74	94.55	89.11	89.69	73.19
SD 	8.51 	10.29	8.41	7.10	10.0	8.63	7.87	7.71
SR 	6.45 	5.40	7.30	10.38	9.42	10.33	11.39	9.49
